In [2]:
import sqlite3

# S path to .db file
db_file = 'xeneta.db'

# Connect to the database
connection = sqlite3.connect(db_file)

# Create a cursor object
cursor = connection.cursor()



Extracting mapping of port names and port codes from SQLite db

In [3]:
import pandas as pd

# Query the database
query = "SELECT * FROM ports"
results = connection.execute(query).fetchall()

# Close the connection
connection.close()

# Create a DataFrame from the query results
df = pd.DataFrame(results, columns=['port_name', 'port_code'])

# Drop duplicate rows
lookup = df.drop_duplicates()


In [4]:
lookup

,port_name,port_code
0,"Anqing, China",CNAQG
1,"Arkhangelsk, Russia",RUARH
2,"Bangkok, Thailand",THBKK
3,"Batangas, Philippines",PHBTG
4,"Beihai, China",CNBIH
...,...,...
142,"Zhanjiang, China",CNZNG
143,"Zhaoqing, China",CNZQG
144,"Zhenjiang, China",CNZHE
145,"Zhongshan, China",CNZSN


In [27]:
# Read the ocean freight rates sheet
ocean_rates = pd.read_excel('data_science_test_1.xlsx', sheet_name='Ocean Freight Rates',skiprows=6)

# Read lookup table
port_lookup = lookup

# Load the static information
static_info = pd.read_excel('data_science_test_1.xlsx', sheet_name='Ocean Freight Rates', nrows=5, usecols=[0, 1])  # Load the first two columns of first 6 rows
static_dict = dict(zip(static_info.iloc[:, 0], static_info.iloc[:, 1]))  # Convert to dictionary

Handle Ports:Assign Origin Port and Destination Port based on priority (Load/Discharge over Receipt/Delivery):

In [28]:
def determine_origin(row):
    return row["Load Port"] if pd.notna(row["Load Port"]) else row["Receipt"]

def determine_destination(row):
    return row["Discharge Port"] if pd.notna(row["Discharge Port"]) else row["Delivery"]

ocean_rates["Origin Port"] = ocean_rates.apply(determine_origin, axis=1)
ocean_rates["Destination Port"] = ocean_rates.apply(determine_destination, axis=1)

for column, value in static_dict.items():
    ocean_rates[column] = value

# Merge lookup table to get port codes
def add_port_codes(rates_df, port_lookup_df, port_column, code_column):
    return rates_df.merge(
        port_lookup_df.rename(columns={"port_name": port_column, "port_code": code_column}),
        left_on=port_column,
        right_on=port_column,
        how="left",
    )

# Add Origin and Destination Port codes
ocean_rates = add_port_codes(ocean_rates, port_lookup, "Receipt", "Origin Port (code)")
ocean_rates = add_port_codes(ocean_rates, port_lookup, "Delivery", "Destination Port (code)")

In case OHC/DHC values are given as 0, and assuming that they are forgotten to be included in the 'Inclusive Surcharges' column.

In [29]:

inclusive_surcharges_condition = (
    (ocean_rates["Charge"].isin(['OHC', 'DHC'])) &
    (ocean_rates["20DRY"] == '0 USD') &
    (ocean_rates["40DRY"] == '0 USD') &
    (ocean_rates["40HDRY"] == '0 USD') &
    ~ocean_rates.apply(lambda row: row['Charge'] in row['Inclusive Surcharges'], axis=1)  # Element-wise comparison
)

# Append the charge to 'Inclusive Surcharges' column if condition is met
ocean_rates.loc[inclusive_surcharges_condition, 'Inclusive Surcharges'] = ocean_rates.apply(
    lambda row: f"{row['Inclusive Surcharges']}, {row['Charge']}" 
    if pd.notnull(row['Inclusive Surcharges']) 
    else row['Charge'], axis=1
)

In [30]:


# THC Used Calculation function

def calculate_thc(row):
    inclusive = row["Inclusive Surcharges"]
    # Check if OHC and DHC are zero and adjust the inclusion logic
    if "OHC" not in inclusive and row["Charge"] == 0:
        inclusive += " OHC"
    if "DHC" not in inclusive and row["Charge"] == 0:
        inclusive += " DHC"
    
    includes_ohc = "OHC" in inclusive
    includes_dhc = "DHC" in inclusive
    
    if includes_ohc and not includes_dhc:
        return "OTHC"
    elif includes_dhc and not includes_ohc:
        return "DTHC"
    elif includes_ohc and includes_dhc:
        return "BOTH"
    else:
        return "NONE"

In [32]:
#  rename equipment types
def prioritize_ports(row):
    return row["Load Port"] if pd.notnull(row["Load Port"]) else row["Receipt"]

def rename_equipments(col):
    mapping = {
        "20DRY": "20DC",
        "40DRY": "40DC",
        "40HDRY": "40HC",
        "45HDRY": "45HQ",
    }
    return mapping.get(col, col)


#filter dataframe

ocean_rates = ocean_rates[(ocean_rates["Commodity Name"] == "FAK") &
                          (ocean_rates["Service Mode"].isin(["CY/CY", "CYCY","CY / CY"])) &
                          (ocean_rates["Rate Basis"] == "PER_CONTAINER")]

ocean_rates.rename(columns=rename_equipments, inplace=True)
ocean_rates = ocean_rates.drop(columns=["45HQ"], errors="ignore").drop_duplicates()

ocean_rates["THC Used"] = ocean_rates.apply(calculate_thc, axis=1)


# dropping rows If none of the datapoints have any non-zero price for that charge/surcharge 
# note: I am not sure if i understood this part to be honest. I am assuming this is what was implied?

ocean_rates = ocean_rates.loc[
    ~((ocean_rates["20DC"] == '0 USD') & 
      (ocean_rates["40DC"] == '0 USD') & 
      (ocean_rates["40HC"] == '0 USD')), 
    :
]


In [44]:
def process_charges(df):
    # List all unique charges in the Charge column
    charges = df["Charge"].unique()

    # Loop through all unique charges
    for charge in charges:
        for col in ["20DC", "40DC", "40HC"]:
            # Create corresponding currency columns for each charge type
            currency_col_name = f"{charge}, Currency"  # e.g., "BAS, Currency"
            
            # Extract currency from a valid string sample
            sample_value = df.loc[df["Charge"] == charge, col].iloc[0]
            if isinstance(sample_value, str) and ' ' in sample_value:
                currency = sample_value.split(' ')[1]
            else:
                currency = None

            # Add the currency column
            df[currency_col_name] = df.apply(
                lambda row: currency if row["Charge"] == charge else None, axis=1
            )

            # Create dynamic columns for charge type with numeric price only
            col_name = f"{charge}, {col}"  # e.g., "BAS, 20DC"
            df[col_name] = df.apply(
                lambda row: extract_price(row[col]) if row["Charge"] == charge else None, axis=1
            )

    return df


def extract_price(value):
    """
    Extracts the numeric price from a value. 
    Handles cases where value is a string with a currency or a numeric value.
    """
    if isinstance(value, str) and ' ' in value:
        try:
            return float(value.split(' ')[0])
        except ValueError:
            return None  # Handle unexpected formats gracefully
    elif isinstance(value, (int, float)):  # If already numeric
        return value
    return None  # For other cases (e.g., NaN, None)

ocean_rates = process_charges(ocean_rates)

In [45]:

# Ensure columns are renamed for alignment with the desired format
final_ocean_rates = ocean_rates.rename(
    columns={
        "Effective Date": "Rate - Valid from",
        "Expiry Date": "Rate - Valid to",
        "Receipt": "Origin Port (name)",
        "Delivery": "Destination Port (name)",
        "Customer Name": "Customer",
        "Carrier": "Carrier",
        "Contract Number": "Contract Number",
        "Commodity Name": "Commodity"
    }
)

final_ocean_rates = final_ocean_rates.drop(columns=["Load Port", "Discharge Port", "Inclusive Surcharges","Charge", "Service Mode","Rate Basis", "Last Acceptance Date","Origin Port","Destination Port", "20DC","40DC","40HC"])

desired_col_order = ['Rate - Valid from',	'Rate - Valid to',	'Origin Port (name)',	'Origin Port (code)',	'Destination Port (name)',	'Destination Port (code)',	'Customer',	'Carrier',	'Contract Number','Commodity', 'THC Used']

reordered_columns  = desired_col_order + [col for col in final_ocean_rates.columns if col not in desired_col_order]

final_ocean_rates = final_ocean_rates[reordered_columns]

final_ocean_rates.to_excel('output.xlsx', index=False)